# Multilayer perceptron classification using *scikit-learn*

In this notebook we will use a multilayer perceptron to classify a subset of the *MNIST* (Modified National Institute of Standards and Technology) database of handwritten digits (http://yann.lecun.com/exdb/mnist/ ). A multilayer perceptron is a feedforward artificial neural network.


The basic steps for model fitting and testing are the same as for *kNN*, except for neural networks you should always scale your data.

1. Scale the data so each feature is on the same scale
2. Create the estimator or model, using *MLPClassifier*
3. Train the model using model.train()
4. Make predictions using model.predict()



## Loading and understanding the data

In [ ]:
from sklearn import datasets
digits = datasets.load_digits()

To demonstrate how the multilayer perceptron works, we will look only at the digits 0, 1, and 7

In [ ]:
import numpy as np
keep = np.logical_or(digits.target <= 1, digits.target == 7) 

digits.data = digits.data[keep,]
digits.target = digits.target[keep]
digits.images = digits.images[keep,]

Stores the features in *X* and the target values in _y_

In [ ]:
X = digits.data
y = digits.target

The feature matrix _X_ is a 2D array with dimensions 539 x 64; each row contains the 64 features for a sample, which is the 'flattened' version of the 8x8 image; *y* contains the 539 target values (for the values 0, 1, and 7).

In [ ]:
print('Shape of X is: ', X.shape)
print('Shape of y is: ', y.shape)

Let's visualize the first 30 numbers:

In [ ]:
import matplotlib.pyplot as plt

# set up the plot
figure, axes = plt.subplots(3,10, figsize = (15,6))

for ax,image,number in zip(axes.ravel(), digits.images, y) :
    ax.axis('off')
    ax.imshow(image, cmap = plt.cm.gray_r)
    ax.set_title('Number: ' + str(number))

## Split the data into training and testing datasets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=17, stratify = y)

## Scale the data

Neural networks perform better if all features are on the same scale. We use the *StandardScaler* which scales each feature to have a mean (average) of 0 and a variance of 1.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# fit the scaler on the training data
scaler.fit(X_train)

# then user the scaler to scale the training and testing data 
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Each feature now has mean 0 and variance 1 (unless all training values are the same, in which case the variance will be 0):

In [ ]:
# look at first 5 column means and variances:
import pandas as pd
Xmeans = X_train.mean(axis = 0)[:5]
Xvars = X_train.var(axis = 0)[:5]
df = pd.DataFrame( {'mean': Xmeans, 'var': Xvars} )
df.round(2)

### Fit the model

We can train a neural network using the Multilayer perceptron classifier (*MLPClassifier*) from *scikit-learn*.

For this classifier we specify the following:

- hidden_layer_sizes: a single value for the size of the hidden a layer, or a list of values for the sizes of multiple hidden layers
- max_iter: the maximum number of epochs (the number of times each data point is used)
- verbose: set to True to print progress

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=4, max_iter=1000, verbose = True, random_state=211)
mlp.fit(X_train, y_train)

We can plot the loss function to see how well our model is learning. If the loss curve should stabilize at a lower value; if not, then adjustments should be made (such as having more epochs, increasing the amount of training data, or changing some of the training model parameters).

In [ ]:
import matplotlib.pyplot as plt
plt.plot(mlp.loss_curve_)
plt.xlabel('Number of epochs')
plt.ylabel('loss')
plt.title('The loss curve for our mlp classifier')
None

### Make predictions in the *test* dataset

In [ ]:
y_pred = mlp.predict(X_test)

### Evaluate the results by generating a *classification report*  which calculates various performance measures

Our *multilayer perceptron* classifier correctly identifies 0,1, and 7 (nearly) 100% of the time.

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print(report)

## Evaluate the results by looking at the *confusion matrix*

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
confusion = confusion_matrix(y_true = y_test, y_pred = y_pred)
df = pd.DataFrame(confusion, columns= [0,1,7], index = [0,1,7])
s = sns.heatmap(df, annot = True, cmap = 'nipy_spectral_r', )
s.set_title('Confusion matrix for MNIST dataset')
None

## Let's understand the structure of our neural network

We can get the weights and biases from 
- *mlp.coefs_*:  a list containing the weight matrices, containing the weight of each input for each node in the layer
- *mlp.intercepts_*: a list of the biases for each layer (every node in a layer has a bias term)

In [ ]:
for i,c in enumerate(mlp.coefs_) :
    print('weight matrix for layer #', i + 1, ': ', c.shape, sep = '')

Our neural network has the following structure:
- an input layer consisting of 64 nodes (the 64 features, representing the 8x8 grid of pixels for a sample)
- a hidden layer consisting of 4 nodes
- an output layer consisting of 3 nodes (corresponding to the probabilities that the sample is a 0,1, or 7, respectively)



The *mlp.predict* method can be used to make a prediction, and get the predicted value. However this prediction is based on comparing the predicted probabilities for each class, and selecting the class with the maximum probability. This is illustrated below.

In [ ]:
preds = mlp.predict(X_test)
pred_probs = mlp.predict_proba(X_test).round(2)

l = [ [num, *matrix] for num, matrix in zip(preds[:3], pred_probs[:3])]
pd.DataFrame(l, columns = ['predicted value', 'prob(0)', 'prob(1)', 'prob(7)'])

### Visualizing the weights of the neural network

In order to understand how the neural network works, let's visualize the weight matrix for each of the 4 neurons in the hidden layer. Darker values indicate larger weights, and inputs with higher weights will have a larger effect on the output.

In [ ]:
fig, axes = plt.subplots(1, 4)
for coef, ax in zip(mlp.coefs_[0].T, axes.ravel()):
    ax.matshow(coef.reshape(8, 8), cmap=plt.cm.gray_r)#, vmin=.5 * vmin,vmax=.5 * vmax)
    ax.set_xticks(())
    ax.set_yticks(())

Each hidden node captures one or more features (local structures) that help classify the number.
The code below looks at the weights applied to inputs from the hidden layer to the *output* layer. This shows how activation of the hidden nodes influences the prediction. For example, if the first hidden node is activated, what number (0,1, or 7) would be predicted?

In [ ]:
pd.DataFrame(mlp.coefs_[1].round(2), index = ['hidden node 1', 'hidden node 2', 'hidden node 3', 'hidden node 4'],
            columns = ['prob(0)', 'prob(1)', 'prob(7)'])